In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ["SM_FRAMEWORK"] = "tf.keras"

import sys
sys.path.append('../src/')
from itertools import product
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard, CSVLogger
import segmentation_models as sm
import keras
import numpy as np
import matplotlib.pyplot as plt

from data.dataloader import Dataloder
from data.dataset import Dataset
from data.data_utils import get_validation_augmentation,get_preprocessing,get_training_augmentation
from visualization.viz_utils import visualize, denormalize
import json


keras.utils.set_random_seed(812)

2024-09-23 00:34:58.186579: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-23 00:34:58.212197: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-23 00:34:58.219709: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-23 00:34:58.237602: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-23 00:34:59.989416: W tensorflow/compiler/tf2

Segmentation Models: using `tf.keras` framework.


/home/semcovici/pesquisa/segmentation-of-blood-vessels-in-retinal-images/.venv/lib/python3.11/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.16 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
DATA_DIR = '../data/raw/'
PATH_MODELS = "../models/"
PATH_LOGS = "../logs/"
RESULTS_PATH = "../results/"

# define os paths dos dados
x_train_dir = os.path.join(DATA_DIR, 'training', 'input')
y_train_dir = os.path.join(DATA_DIR, 'training', 'target')

x_test_dir = os.path.join(DATA_DIR, 'test', 'input')
y_test_dir = os.path.join(DATA_DIR, 'test', 'target')


# Gerar lista de IDs
train_ids = ['%02d' % i for i in range(21, 41)]
test_ids = ['%02d' % i for i in range(1, 21)]

# Dividir IDs de treinamento em conjuntos de treinamento e validação
val_ids = train_ids[-4:]
train_ids = train_ids[:-4]

In [3]:
BACKBONE = 'efficientnetb3'
BATCH_SIZE = 4
LR = 0.0001
EPOCHS = 3
n_classes = 1

# Definir otimizador
optim = keras.optimizers.Adam(LR)

# Definir perda e métricas
dice_loss = sm.losses.DiceLoss()
focal_loss = sm.losses.BinaryFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

preprocess_input = sm.get_preprocessing(BACKBONE)
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5), 'accuracy', 'precision', 'recall']

2024-09-23 00:35:02.860565: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_SYSTEM_DRIVER_MISMATCH: system has unsupported display driver / cuda driver combination
2024-09-23 00:35:02.860609: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:135] retrieving CUDA diagnostic information for host: pop-os
2024-09-23 00:35:02.860615: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:142] hostname: pop-os
2024-09-23 00:35:02.860783: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:166] libcuda reported version is: 560.35.3
2024-09-23 00:35:02.860808: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:170] kernel reported version is: 555.58.2
2024-09-23 00:35:02.860813: E external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:252] kernel version 555.58.2 does not match DSO version 560.35.3 -- cannot find working devices in this configuration


In [7]:
activation_list = [
    'sigmoid', 
    # 'softmax'
            ]
augmentation_list=[
    # False,
   True
    ]

In [8]:
for activation, augmentation in product(activation_list, augmentation_list):

    model_name = f'model_{BACKBONE}_{activation}_aug={augmentation}'
    
    valid_aug = None
    train_aug = None
    
    
    if augmentation:
        valid_aug = get_validation_augmentation()
        train_aug = get_training_augmentation()
        
    # Criar modelo
    model = sm.Unet(BACKBONE, classes=n_classes, activation=activation)
    # Compilar modelo
    model.compile(optimizer=optim, loss=total_loss, metrics=metrics)
    
    preprocessing = get_preprocessing(preprocess_input)


    # Dataset para imagens de treinamento
    train_dataset = Dataset(
        x_train_dir,
        y_train_dir,
        ids=train_ids,
        dataset_name='training',
        augmentation=train_aug,
        preprocessing=preprocessing,
    )


    # Dataset para imagens de validação
    valid_dataset = Dataset(
        x_train_dir,
        y_train_dir,
        ids=val_ids,
        dataset_name='training',
        augmentation=valid_aug,
        preprocessing=preprocessing,
    )

    # Dataset para imagens de teste
    test_dataset = Dataset(
        x_test_dir,
        y_test_dir,
        ids=test_ids,
        dataset_name='test',
        augmentation=valid_aug,
        preprocessing=preprocessing,
    )

    train_dataloader = Dataloder(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    valid_dataloader = Dataloder(valid_dataset, batch_size=1, shuffle=False)
    test_dataloader = Dataloder(test_dataset, batch_size=1, shuffle=False)

    # # Verificar formas para erros
    assert train_dataloader[0][0].shape == (BATCH_SIZE, 256, 256, 3)
    assert train_dataloader[0][1].shape == (BATCH_SIZE, 256, 256, n_classes)



    model_weights = PATH_MODELS + f'weights/best_{model_name}.weights.h5'

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=10),
        ModelCheckpoint(PATH_MODELS + f'model_file/best_{model_name}.keras', save_best_only=True, monitor='val_loss'),
        ModelCheckpoint(model_weights, save_weights_only=True, save_best_only=True, mode='min'),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5),
        TensorBoard(log_dir=PATH_LOGS),
        CSVLogger(PATH_MODELS + f'training_log_{model_name}.csv')
    ]

    # Treinar modelo
    history = model.fit(
        train_dataloader,
        steps_per_epoch=len(train_dataloader),
        epochs=EPOCHS,
        callbacks=callbacks,
        validation_data=valid_dataloader,
        validation_steps=len(valid_dataloader),
    )
    
    json.dump(history.history, open(RESULTS_PATH + f"history/history_{model_name}.json", "w"))

Epoch 1/3


/home/semcovici/pesquisa/segmentation-of-blood-vessels-in-retinal-images/.venv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 120s 9s/step - accuracy: 0.8073 - f1-score: 0.1595 - iou_score: 0.0868 - loss: 0.9280 - precision: 0.1293 - recall: 0.2069 - val_accuracy: 0.8820 - val_f1-score: 0.1317 - val_iou_score: 0.0705 - val_loss: 0.9133 - val_precision: 0.1715 - val_recall: 0.1073 - learning_rate: 1.0000e-04
Epoch 2/3


2024-09-23 00:39:06.299762: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/usr/lib/python3.11/contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
2024-09-23 00:39:06.377407: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.0000e+00 - f1-score: 0.0000e+00 - iou_score: 0.0000e+00 - loss: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 3/3


/home/semcovici/pesquisa/segmentation-of-blood-vessels-in-retinal-images/.venv/lib/python3.11/site-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,f1-score,iou_score,loss,precision,recall
  current = self.get_monitor_value(logs)
/home/semcovici/pesquisa/segmentation-of-blood-vessels-in-retinal-images/.venv/lib/python3.11/site-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)
/home/semcovici/pesquisa/segmentation-of-blood-vessels-in-retinal-images/.venv/lib/python3.11/site-packages/keras/src/callbacks/callback_list.py:96: UserWarning: Learning rate reduction is conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,f1-score,iou_score,loss,precision,recall,learning_rate.
  callback.on_epoch_end(epoch, logs

4/4 ━━━━━━━━━━━━━━━━━━━━ 22s 6s/step - accuracy: 0.8414 - f1-score: 0.2515 - iou_score: 0.1442 - loss: 0.9001 - precision: 0.2084 - recall: 0.3183 - val_accuracy: 0.8695 - val_f1-score: 0.2459 - val_iou_score: 0.1402 - val_loss: 0.9127 - val_precision: 0.2381 - val_recall: 0.2547 - learning_rate: 1.0000e-04


# Avaliação

In [ ]:
# # Avaliação do Modelo

# Carregar os melhores pesos
model.load_weights(model_weights)

# Avaliar no conjunto de teste
scores = model.evaluate(test_dataloader)

print("Loss: {:.5}".format(scores[0]))
for metric, value in zip(metrics, scores[1:]):
    print("mean {}: {:.5}".format(metric.__name__ if type(metric) != str else metric, value))

# # Visualização dos resultados no conjunto de teste

for i in range(5):
    image, gt_mask = test_dataset[i]
    image = np.expand_dims(image, axis=0)
    pr_mask = model.predict(image).round()
    
    visualize(
        image=denormalize(image.squeeze()),
        ground_truth_mask=gt_mask.squeeze(),
        predicted_mask=pr_mask.squeeze(),
    )